# 4.3.2 有価証券報告書内のMD＆Aが記載されているhtmファイルを抽出　

　Zipファイルで送られてきた大量の有価証券書のデータをPythonにより自動的に解凍した。その後、有価証券報告書内のMD＆A情報のみが記載されたhtmファイルのみを抽出をした。

In [60]:
import pandas as pd
import glob
import os
import zipfile
from datetime import datetime as dt
import bs4 
import re

In [61]:
def make_industry_zip_list_hash(data_frame, data_frame_name):
    
    industry_zip_list_hash = {}
    industry_li = make_type_of_industry_list(data_frame)
    zip_files = call_zip_files(data_frame_name)
    for industry in industry_li :
        industry_zip_list_hash[industry]= list(filter(lambda x: industry in x , zip_files))
    
    return   industry_zip_list_hash
    

def make_type_of_industry_list(data_frame : pd.DataFrame, industry_col="[業種（東証）]"):
    return list(set(data_frame[industry_col]))


def call_zip_files(data_frame_name):
    zip_files = glob.glob(f"../**/SampleData/{data_frame_name}/**/**.zip", recursive=True)
    return zip_files

def call_unziped_htm_files_dir(data_frame_name: str):

    unziped_htm_files_dir = os.getcwd()+  "/UnzipedHtmFiles"
    if not os.path.exists(unziped_htm_files_dir) :
        os.mkdir(unziped_htm_files_dir)

    unziped_htm_files_dir_with_df_name = unziped_htm_files_dir + f"/{data_frame_name}"
    if not os.path.exists(unziped_htm_files_dir_with_df_name) :
        os.mkdir(unziped_htm_files_dir_with_df_name)
    
    return unziped_htm_files_dir_with_df_name


#--------------------------------------------------------------------
def unzip_html_to_unziped_htm_files_dir(industry_zip_list_hash, filepath_unziped):
    
    sum_files_len = sum(map(len, industry_zip_list_hash.values()))
    zip_files_len = 1
    
    for industy_name, zip_files in industry_zip_list_hash.items() :
        zip_files_len += len(zip_files)
        
        industry_dir = call_industry_dir(filepath_unziped, industy_name)
    
        for idx ,zip_file in enumerate(zip_files):
            try:
                with zipfile.ZipFile(zip_file) as existing_zip:

                    candidate_files = existing_zip.namelist()

                    for c_f in candidate_files:
                        basename = os.path.basename(c_f)
                        #第２部のもの(MD&Aが記載されている見出しの部分)を判定
                        if re.match(r'01020.*htm', basename) != None :
                            print(c_f)
                            date_str = c_f[-20:-10]
                            date_dt = dt.strptime(date_str, '%Y-%m-%d')

                            existing_zip.extract(c_f, industry_dir)

                    print(f"{idx + 1} / {len(zip_files)} ||  {zip_files_len - 1}/{sum_files_len}")
            except Exception :
                print(zip_file)
                with open("unzipping_html_error.txt", "a") as f:
                    f.write(zip_file + "\n")
                
                
def call_industry_dir(filepath_unziped, industry_name: str):
    industry_dir =  filepath_unziped+  f"/{industry_name}"
    if not os.path.exists(industry_dir) :
        os.mkdir(industry_dir)
    return industry_dir

In [62]:
f = call_zip_files("renketsu") + call_zip_files("hirenketsu")
len_f = len(f)
print(len_f,  len_f/3)

6171 2057.0


In [58]:
#renkestsuとhirenketsuを設定
data_frame_name ="renketsu" 
data_frame = pd.read_csv("/home/jovyan/1CalliingEdinetApi"+f"/EdinetIdxFiles/edinet_{data_frame_name}.csv", skiprows=4)

In [59]:
industry_zip_list_hash = make_industry_zip_list_hash(data_frame, data_frame_name)
filepath_unziped = call_unziped_htm_files_dir(data_frame_name)
unzip_html_to_unziped_htm_files_dir(industry_zip_list_hash, filepath_unziped)

XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E27633-000_2018-03-31_01_2018-06-27_ixbrl.htm
1 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E03867-000_2018-03-31_01_2018-06-22_ixbrl.htm
2 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E00165-000_2018-03-31_01_2018-06-22_ixbrl.htm
3 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01202-000_2018-03-31_01_2018-06-20_ixbrl.htm
4 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04047-000_2018-03-31_01_2018-06-25_ixbrl.htm
5 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E03953-000_2018-03-31_01_2018-06-21_ixbrl.htm
6 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E27759-000_2018-03-31_01_2018-06-27_ixbrl.htm
7 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E03861-000_2018-03-31_01_2018-06-28_ixbrl.htm
8 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E25621-000_201

XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E00926-000_2019-03-31_01_2019-06-27_ixbrl.htm
73 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E31970-000_2019-03-31_01_2019-06-26_ixbrl.htm
74 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E03855-000_2019-03-31_01_2019-06-27_ixbrl.htm
75 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E27759-000_2019-03-31_01_2019-06-26_ixbrl.htm
76 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E03967-000_2019-03-31_01_2019-06-27_ixbrl.htm
77 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E03997-000_2019-03-31_01_2019-06-26_ixbrl.htm
78 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E27281-000_2019-03-31_01_2019-06-27_ixbrl.htm
79 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04044-000_2019-03-31_01_2019-06-27_ixbrl.htm
80 / 138 ||  138/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E03958

12 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01084-000_2019-03-31_01_2019-06-27_ixbrl.htm
13 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01080-000_2019-03-31_01_2019-06-26_ixbrl.htm
14 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E31632-000_2020-03-31_01_2020-06-25_ixbrl.htm
15 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01080-000_2020-03-31_01_2020-06-25_ixbrl.htm
16 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01077-000_2020-03-31_01_2020-06-29_ixbrl.htm
17 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01082-000_2020-03-31_01_2020-06-25_ixbrl.htm
18 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E24050-000_2020-03-31_01_2020-06-25_ixbrl.htm
19 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E00030-000_2020-03-31_01_2020-06-26_ixbrl.htm
20 / 21 ||  159/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-a

61 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01430-000_2019-03-20_01_2019-06-19_ixbrl.htm
62 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01436-000_2019-03-31_01_2019-06-20_ixbrl.htm
63 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01403-000_2019-03-31_01_2019-06-21_ixbrl.htm
64 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01415-000_2019-03-31_01_2019-06-20_ixbrl.htm
65 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01355-000_2019-03-31_01_2019-06-26_ixbrl.htm
66 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01376-000_2019-03-31_01_2019-06-28_ixbrl.htm
67 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01466-000_2019-03-31_01_2019-06-24_ixbrl.htm
68 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01452-000_2019-03-31_01_2019-06-21_ixbrl.htm
69 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcr

131 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01444-000_2020-03-31_01_2020-06-26_ixbrl.htm
132 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01415-000_2020-03-31_01_2020-06-25_ixbrl.htm
133 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01426-000_2020-03-31_01_2020-06-29_ixbrl.htm
134 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01688-000_2020-03-31_01_2020-06-25_ixbrl.htm
135 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01439-000_2020-03-31_01_2020-06-25_ixbrl.htm
136 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01461-000_2020-03-31_01_2020-06-25_ixbrl.htm
137 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01443-000_2020-03-31_01_2020-06-25_ixbrl.htm
138 / 168 ||  327/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01298-000_2020-03-31_01_2020-06-26_ixbrl.htm
139 / 168 ||  327/6171
XBRL/PublicDoc/0102010_ho

...

278 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05306-000_2019-03-31_01_2019-06-28_ixbrl.htm
279 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05056-000_2019-03-31_01_2019-06-27_ixbrl.htm
280 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05205-000_2019-03-31_01_2019-06-26_ixbrl.htm
281 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E21706-000_2019-03-31_01_2019-06-26_ixbrl.htm
282 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E33238-000_2019-03-31_01_2019-06-26_ixbrl.htm
283 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05410-000_2019-03-31_01_2019-06-28_ixbrl.htm
284 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05575-000_2019-03-31_01_2019-06-27_ixbrl.htm
285 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05248-000_2019-03-31_01_2019-06-27_ixbrl.htm
286 / 513 ||  6099/6171
XBRL/PublicDoc/0

352 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05456-000_2020-03-31_01_2020-06-18_ixbrl.htm
353 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E27303-000_2020-03-31_01_2020-06-22_ixbrl.htm
354 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05598-000_2020-03-31_01_2020-06-25_ixbrl.htm
355 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04857-000_2020-03-31_01_2020-06-19_ixbrl.htm
356 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05267-000_2020-03-31_01_2020-06-19_ixbrl.htm
357 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04828-000_2020-03-31_01_2020-06-19_ixbrl.htm
358 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05562-000_2020-03-31_01_2020-06-19_ixbrl.htm
359 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05153-000_2020-03-31_01_2020-06-19_ixbrl.htm
360 / 513 ||  6099/6171
XBRL/PublicDoc/0

426 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E21261-000_2020-03-31_01_2020-06-25_ixbrl.htm
427 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05248-000_2020-03-31_01_2020-06-26_ixbrl.htm
428 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05428-000_2020-03-31_01_2020-06-26_ixbrl.htm
429 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E31381-000_2020-03-31_01_2020-06-25_ixbrl.htm
430 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04837-000_2020-03-31_01_2020-06-25_ixbrl.htm
431 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04707-000_2020-03-31_01_2020-06-26_ixbrl.htm
432 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E33034-000_2020-03-31_01_2020-06-25_ixbrl.htm
433 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04824-000_2020-03-31_01_2020-06-26_ixbrl.htm
434 / 513 ||  6099/6171
XBRL/PublicDoc/0

498 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05593-000_2020-03-31_01_2020-07-01_ixbrl.htm
499 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05425-000_2020-03-31_01_2020-06-30_ixbrl.htm
500 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05737-000_2020-03-31_01_2020-07-03_ixbrl.htm
501 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E04704-000_2020-03-31_01_2020-07-01_ixbrl.htm
502 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05269-000_2020-03-31_01_2020-07-03_ixbrl.htm
503 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E00518-000_2020-03-31_01_2020-07-10_ixbrl.htm
504 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E05524-000_2020-03-31_01_2020-07-17_ixbrl.htm
505 / 513 ||  6099/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E33238-000_2020-03-31_01_2020-07-27_ixbrl.htm
506 / 513 ||  6099/6171
XBRL/PublicDoc/0

59 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E00024-000_2020-03-31_01_2020-06-26_ixbrl.htm
60 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01337-000_2020-03-31_01_2020-06-26_ixbrl.htm
61 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01308-000_2020-03-31_01_2020-06-26_ixbrl.htm
62 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E00021-000_2020-03-31_01_2020-06-30_ixbrl.htm
63 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01352-000_2020-03-31_01_2020-06-26_ixbrl.htm
64 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E00023-000_2020-03-31_01_2020-06-26_ixbrl.htm
65 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01336-000_2020-03-31_01_2020-06-29_ixbrl.htm
66 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcrp030000-asr-001_E01331-000_2020-03-31_01_2020-06-29_ixbrl.htm
67 / 72 ||  6171/6171
XBRL/PublicDoc/0102010_honbun_jpcr